<a href="https://colab.research.google.com/github/skywalker0803r/oil_web/blob/master/%E9%96%8B%E7%99%BC%E9%81%8E%E7%A8%8B%E6%96%87%E4%BB%B6/%E7%AC%AC%E5%9B%9B%E7%89%88%E6%A8%A1%E7%B5%84%E9%96%8B%E7%99%BC_SVR_%E5%BA%8F%E5%88%97%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.svm import SVR
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
import warnings;warnings.simplefilter('ignore')
from sklearn.pipeline import Pipeline
import joblib
from scipy.special import softmax
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load raw data

In [0]:
train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/train_4565.csv',index_col=0)
test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/test_170.csv',index_col=0)

# data for modeling

In [3]:
cols = train.columns.tolist()
x_cols = ['T10','T50','T90','N+A']
y_cols = ['C5NP','C5IP','C5N','C6NP','C6IP','C6N','C6A','C7NP','C7IP','C7N','C7A',
'C8NP','C8IP','C8N','C8A','C9NP','C9IP','C9N','C9A','C10NP','C10IP','C10N','C10A']
X_train = train[x_cols]
X_test = test[x_cols]
y_train = train[y_cols]
y_test = test[y_cols]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4565, 4)
(4565, 23)
(170, 4)
(170, 23)


# define model

In [0]:
class custom_model(object):
  def __init__(self,x_cols,y_cols):
    self.x_cols = x_cols
    self.y_cols = y_cols
    self.model_23 = {}
    for y_name in y_cols:
      self.model_23[y_name] = Pipeline([('scaler',StandardScaler()),('reg',SVR(C=0.3))])
  
  def feature_engineering(self,X):
    #X['f1'] = X['T90']-X['T50']
    #X['f2'] = X['T50']-X['T10']
    return X
  
  def fit(self,X,y):
    X = self.feature_engineering(X)
    for y_name in tqdm(self.y_cols):
      self.model_23[y_name].fit(X,y[y_name])
      y_pred = self.model_23[y_name].predict(X)
      # 序列預測把y_pred併入X 
      X.loc[:,y_name] = y_pred
    #變回來
    X = X[self.x_cols]
  
  def predict(self,X):
    results = pd.DataFrame(index=[*range(len(X))],columns=self.y_cols)
    X = self.feature_engineering(X)
    for y_name in self.y_cols:
      y_pred = self.model_23[y_name].predict(X)
      results.loc[:,y_name] = y_pred
      # 序列預測把y_pred併入X
      X.loc[:,y_name] = y_pred
    #變回來
    X = X[self.x_cols]
    return results.values

# fit model

In [5]:
model = custom_model(x_cols,y_cols)
model.fit(X_train,y_train)

# model predict

In [6]:
y_pred = model.predict(X_test)
y_pred.shape

(170, 23)

# metric

In [7]:
N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
P_col = ['C5NP','C5IP','C6NP','C6IP','C7NP','C7IP','C8NP','C8IP','C9NP','C9IP','C10NP','C10IP']
print(len(N_col))
print(len(P_col))

11
12


In [8]:
X_test['P'] = 100 - X_test['N+A']
X_test

,T10,T50,T90,N+A,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,P
0,98.6,119.2,146.1,40.133,1.190894,0.807816,0.160609,3.573410,2.227260,3.516232,0.654154,7.918605,6.272781,9.455625,3.257780,7.923163,8.302073,6.966316,5.818806,5.146293,9.245465,4.937084,3.466927,1.127751,5.198255,0.668779,0.687152,59.867
1,105.4,122.2,148.2,32.175,0.137756,0.116131,0.102770,3.273672,1.100118,2.163449,0.398651,10.462490,7.612349,6.337935,2.885223,9.681833,10.256803,4.981778,5.586067,6.757531,10.725721,4.448437,3.891992,1.188590,6.041288,0.597686,0.706335,67.825
2,105.8,122.7,149.2,31.428,0.129679,0.107544,0.105217,3.252565,1.090545,2.095699,0.370173,10.462874,7.574888,6.133792,2.822014,9.715474,10.298116,4.816792,5.483437,6.841115,10.733315,4.356014,3.957389,1.313952,6.230529,0.602698,0.854748,68.572
3,105.4,122.4,147.6,31.390,0.116662,0.105996,0.103472,3.401498,1.171461,2.103547,0.412287,10.469417,7.474605,6.069143,2.846285,9.805304,10.347694,4.833161,5.573599,6.879981,10.817638,4.367383,3.941348,1.197543,6.175576,0.543607,0.638214,68.610
4,105.4,122.6,148.2,32.418,0.150804,0.123865,0.102519,3.239423,1.122442,2.139366,0.405910,10.306411,7.425585,6.255281,2.871297,9.636309,10.172680,4.979009,5.660119,6.796300,10.771386,4.415727,4.008064,1.231850,6.220064,0.584412,0.733545,67.582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,97.8,117.1,144.5,53.841,1.238342,0.813451,0.202284,2.589025,2.482650,5.140911,0.580152,4.984934,4.202473,15.461110,3.532293,6.745408,6.304790,10.510846,6.439827,3.600141,7.354657,5.816599,3.198371,0.985047,4.169442,0.805335,0.561651,46.159
166,96.6,115.9,141.7,53.442,1.370203,0.918452,0.227366,2.756401,2.763571,5.331270,0.579610,5.016893,4.186057,15.569645,3.491206,6.882085,6.441221,10.459182,6.458633,3.645366,7.380047,5.963470,3.037378,0.742235,3.937879,0.701433,0.403410,46.558
167,94.4,114.9,141.6,52.549,1.731233,1.165095,0.260432,3.065380,3.048210,5.543163,0.586828,5.083419,4.421393,15.691269,3.430610,6.719553,6.427086,10.293337,6.197018,3.562517,7.215538,5.952469,2.903807,0.718021,3.891205,0.660860,0.407140,47.451
168,91.6,113.4,145.2,49.827,2.315809,1.424868,0.286699,3.402958,3.149375,5.672432,0.570165,5.630333,5.512768,16.004519,3.361137,5.999161,6.333739,9.448510,5.057458,3.220685,6.934163,5.327789,2.528834,1.059260,4.314302,0.826560,0.915687,50.173


In [0]:
def normalize(x):
  x = x/x.sum(axis=1).values.reshape(-1,1)
  return x

In [10]:
y_pred = pd.DataFrame(y_pred,index=[*range(len(X_test))],columns=y_cols)
y_pred[N_col] = normalize(y_pred[N_col])*X_test['N+A'].values.reshape(-1,1)
y_pred[P_col] = normalize(y_pred[P_col])*X_test['P'].values.reshape(-1,1)
y_pred.to_excel('test_170_y_pred.xlsx')
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.209752,0.820608,0.162814,3.629996,2.262529,3.564507,0.663135,8.043998,6.372113,9.585444,3.302507,8.048629,8.433539,7.061958,5.898694,5.227786,9.391870,5.004867,3.514526,1.145609,5.280571,0.677961,0.696586
1,0.138718,0.116942,0.103009,3.296550,1.107806,2.168482,0.399578,10.535609,7.665550,6.352680,2.891935,9.749496,10.328485,4.993367,5.599062,6.804757,10.800680,4.458786,3.901046,1.196897,6.083509,0.599076,0.707979
2,0.131251,0.108848,0.104651,3.291999,1.103766,2.084426,0.368182,10.589725,7.666725,6.100797,2.806833,9.833264,10.422970,4.790881,5.453940,6.924056,10.863445,4.332582,3.936101,1.329882,6.306067,0.599456,0.850151
3,0.117772,0.107004,0.103334,3.433861,1.182606,2.100733,0.411735,10.569027,7.545721,6.061024,2.842478,9.898595,10.446146,4.826696,5.566143,6.945439,10.920560,4.361541,3.936076,1.208936,6.234332,0.542879,0.637361
4,0.151668,0.124574,0.103356,3.257977,1.128871,2.156847,0.409227,10.365443,7.468116,6.306395,2.894759,9.691502,10.230945,5.019695,5.706370,6.835227,10.833081,4.451810,4.040815,1.238906,6.255690,0.589187,0.739539


In [11]:
y_test.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


In [12]:
temp = pd.DataFrame()
temp['real_N+A'] = X_test['N+A'].values
temp['pred_N+A'] = y_pred[N_col].sum(axis=1).values
temp

,real_N+A,pred_N+A
0,40.133,40.133
1,32.175,32.175
2,31.428,31.428
3,31.390,31.390
4,32.418,32.418
...,...,...
165,53.841,53.841
166,53.442,53.442
167,52.549,52.549
168,49.827,49.827


In [0]:
def mape(a, b):
    a = np.array(a)
    b = np.array(b)
    mask = a != 0
    return (np.fabs(a-b)/a)[mask].mean()

def metric(y_t,y_p,name):
  res = {'R2' : max(r2_score(y_t,y_p), 0),
         'RMSE': sqrt(mean_squared_error(y_t,y_p)),
         'MAPE': mape(y_t,y_p)}
  return pd.DataFrame(res,index=[name])

In [0]:
res = pd.DataFrame()
for y_name in y_cols:
  res = res.append(metric(y_test[y_name],y_pred[y_name],y_name))
res.loc['AVG'] = res.mean()

In [15]:
res.style.background_gradient()

,R2,RMSE,MAPE
C5NP,0.871972,0.305779,0.302849
C5IP,0.738419,0.261722,0.60706
C5N,0.791745,0.0624729,2.21142
C6NP,0.277911,0.343793,0.0633327
C6IP,0.882919,0.357659,0.0931492
C6N,0.886192,0.558672,0.0523298
C6A,0.526214,0.117599,0.170503
C7NP,0.97256,0.364311,0.0429329
C7IP,0.854131,0.422033,0.0487745
C7N,0.982764,0.547111,0.033794


# check 是否 pass

In [16]:
c1 = (len(res.loc[res.RMSE > 0.6]) == 0)
c2 = (res.loc['AVG','RMSE'] < 0.4)
print(c1,c2)

True True


# model save

In [17]:
joblib.dump(model,'model.pkl')
print('save done')

save done
